In [17]:
import math
import numpy as np
import networkx as nx
import matplotlib.pyplot as plt
import time
from tqdm import tqdm

In [18]:
from utils import read_test1_data
from utils import gen_graph
from utils import prepare_synthetic
from utils import shuffle_graph
from utils import preprocessing_data
from utils import get_pairwise_ids

from utils import prepare_test1
from utils import top_n_acc

In [19]:
RANDOM_STATE = 11
SYNTHETIC_NUM = 100
# SYNTHETIC_NUM = 1000

# number of gen nodes
# NUM_MIN = 4000
# NUM_MAX = 4001
NUM_MIN = 200
NUM_MAX = 201


MAX_EPOCHS = 10000
LEARNING_RATE = 1e-4
EMBEDDING_SIZE = 128
DEPTH = 5
BATCH_SIZE = 16

TEST1_NUM = 30

## Read Graph

In [20]:
test1_g, test1_bc, test1_edgeindex = read_test1_data(0)

## Generate Synthetic Graph

In [21]:
train_g = gen_graph(500, 501)
print(len(train_g.edges()))

1984


In [22]:
# [train_g.degree(i) for i in range(train_g.number_of_nodes())]

In [23]:
# nx.betweenness_centrality(train_g)

In [24]:
(np.array(list(train_g.edges())) + 100)[:10]

array([[100, 104],
       [100, 105],
       [100, 106],
       [100, 107],
       [100, 108],
       [100, 109],
       [100, 111],
       [100, 114],
       [100, 129],
       [100, 131]])

In [25]:
# nx.betweenness_centrality(train_g)

## DrBC

In [26]:
from scipy import stats
# from model1 import DrBC
from model import DrBC
import torch
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [27]:
model = DrBC().to(device)
optim = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE)
loss_fn = torch.nn.BCEWithLogitsLoss(reduction='sum')

In [28]:
model.parameters

<bound method Module.parameters of DrBC(
  (linear0): Linear(in_features=3, out_features=128, bias=True)
  (gcn): GCNConv()
  (gru): GRUCell(128, 128)
  (mlp): Sequential(
    (0): Linear(in_features=128, out_features=64, bias=True)
    (1): ReLU()
    (2): Linear(in_features=64, out_features=1, bias=True)
  )
)>

In [29]:
# list(model.parameters())[9].grad.data

In [30]:
pm = list(model.parameters())

for i, p in enumerate(pm):
    print(f"pm{i} shape: {p.shape}")

pm0 shape: torch.Size([128, 3])
pm1 shape: torch.Size([128])
pm2 shape: torch.Size([384, 128])
pm3 shape: torch.Size([384, 128])
pm4 shape: torch.Size([384])
pm5 shape: torch.Size([384])
pm6 shape: torch.Size([64, 128])
pm7 shape: torch.Size([64])
pm8 shape: torch.Size([1, 64])
pm9 shape: torch.Size([1])


In [31]:
# list(dict(nx.degree(train_g)).values())
# list(dict(nx.degree(train_g)).values())
# list(dict(nx.betweenness_centrality(train_g)).values())

In [32]:
def validate(model, v_data):
    model.eval()
    total_acc = 0.
    total_kendall = 0.
    for val_X, val_y, val_edge_index in v_data:
        val_X, val_edge_index = val_X.to(device), val_edge_index.to(device)
        
        with torch.no_grad():
            val_y_pred = model(val_X, val_edge_index)

        val_y_pred = val_y_pred.cpu().detach().numpy()
        val_y = val_y.detach().numpy()

        pred_index = val_y_pred.argsort()[::-1]
        true_index = val_y.argsort()[::-1]
        
        acc = top_n_acc(pred_index, true_index)
        kendall_t, _ = stats.kendalltau(val_y_pred, val_y)

        total_acc += acc
        total_kendall += kendall_t

    total_acc /= len(v_data)
    total_kendall /= len(v_data)
    return total_acc, total_kendall
    

def train(model, optim, loss_fn, epochs:int):
    g_list, dg_list, bc_list  = prepare_synthetic(SYNTHETIC_NUM, (NUM_MIN, NUM_MAX))
    v_data = prepare_test1(TEST1_NUM)
    
    ls_metric = []
    batch_cnt = len(g_list) // BATCH_SIZE
    for e in range(epochs + 1):
        model.train()
        g_list, dg_list, bc_list = shuffle_graph(g_list, dg_list, bc_list)
        batch_bar = tqdm(range(batch_cnt))
        batch_bar.set_description(f'Epochs {e:<5}')
        train_loss = 0
        pair_cnt = 0
        for i in batch_bar:
            # batch
            s_index, e_index = i*BATCH_SIZE, (i+1)*BATCH_SIZE
            train_g, train_dg, train_bc = g_list[s_index: e_index], dg_list[s_index: e_index], bc_list[s_index: e_index]
            X, y, edge_index = preprocessing_data(train_g, train_dg, train_bc)
            X, y, edge_index = X.to(device), y.to(device), edge_index.to(device)
            out = model(X, edge_index)

            # pairwise-loss
            s_ids, t_ids = get_pairwise_ids(train_g)
            out_diff = out[s_ids] - out[t_ids]
            y_diff = y[s_ids] - y[t_ids]
            loss = loss_fn(out_diff, torch.sigmoid(y_diff))

            # optim
            optim.zero_grad()
            loss.backward()
            optim.step()

            pair_cnt += s_ids.shape[0]
            train_loss += (loss.item() * s_ids.shape[0])
            if i == (batch_cnt - 1):
                # last batch
                train_loss /= pair_cnt
                batch_bar.set_postfix(loss=train_loss) 

        if e % 50 == 0:
            val_acc, val_kendall = validate(model, v_data)
            ls_metric.append([e, val_acc, val_kendall])
            print(f"Val Acc: {val_acc * 100:.2f} % | Val KendallTau: {val_kendall:.4f}")
        

_ = train(model, optim, loss_fn, 200)

Epochs 0    : 100%|██████████| 6/6 [00:00<00:00, 43.54it/s, loss=1.11e+4]


Val Acc: 87.20 % | Val KendallTau: 0.7299


Epochs 50   : 100%|██████████| 6/6 [00:00<00:00, 42.66it/s, loss=1.11e+4]


Val Acc: 94.27 % | Val KendallTau: 0.4120


Epochs 100  : 100%|██████████| 6/6 [00:00<00:00, 40.99it/s, loss=1.11e+4]


Val Acc: 94.33 % | Val KendallTau: 0.6398


Epochs 150  : 100%|██████████| 6/6 [00:00<00:00, 46.16it/s, loss=1.11e+4]


Val Acc: 94.27 % | Val KendallTau: 0.6652


Epochs 200  : 100%|██████████| 6/6 [00:00<00:00, 48.50it/s, loss=1.11e+4]


Val Acc: 94.27 % | Val KendallTau: 0.6705


In [33]:
# g = _[2]
# g.degree(list(range(99, 105)))

## To-Do List
* (done) loss_fn 再加上 sigmoid
* (done) pairwise 目前跨圖了
* (done) h 要 normalized
* (done) aggregate 改成 MessagePassing
* (done) synthetic graph 後，shuffle graph 的順序
* (done) 加入 Epochs
* Metric: top1, 5, 10
* Metric: kendall tau distance
* wall-clock running time
* test step
* (done) change to leaky relu -> back to relu